# Pipeline Walkthrough Part 1.

## Data Preparation

In this example I will be giving a (mostly) complete walkthrough of the entire modeling pipeline. I will show how the pipeline can be used to quickly and easily develop machine learning models of human behaviors. Starting with a short example video clip, this walkthrough will include the following steps:

1. Seperating audio from video
2. Extracting relevant features from audio and video
3. Producing simple labels (voice activity labels)
4. Choosing appropriate features

In part 2 I will show how to train a model


# Notebook Setup

First, we need to get set up. Right now this package is still under development and hasn't been released as a pip package, so we will be using the source code directly. The first step is to make sure that the package root directory is in our python path:

In [1]:
import sys
import os

# Inserts the package root "modeling-pipeline" into the python path
sys.path.insert(0, os.path.abspath('../..'))

# The following will throw an error if the package hasn't been added to the path properly:
from pipeline.common.file_utils import get_dirs_from_config

# We can use sys.path to see what the path includes:
sys.path

['/home/interactionlab/chrisb/modeling-pipeline',
 '/home/interactionlab/chrisb/modeling-pipeline/examples/walkthrough',
 '/home/interactionlab/anaconda3/envs/modelenv/lib/python38.zip',
 '/home/interactionlab/anaconda3/envs/modelenv/lib/python3.8',
 '/home/interactionlab/anaconda3/envs/modelenv/lib/python3.8/lib-dynload',
 '',
 '/home/interactionlab/anaconda3/envs/modelenv/lib/python3.8/site-packages',
 '/home/interactionlab/anaconda3/envs/modelenv/lib/python3.8/site-packages/IPython/extensions',
 '/home/interactionlab/.ipython']

Now that we can see that the modeling-pipeline is in our path, we can import modules with the following:

`from pipeline.common.file_utils import get_dirs_from_config`

Which will import the get_dirs_from_config function from the file_utils module in the common module in our pipeline package. 

Note: if you make changes to any of the packages **you may need to restart this notebook's kernel** in order for the import to be reloaded with your changes.

# 0. File Manipulation Example (e.g. get_dirs_from_config())

One of the biggest annoyances with working with custom datasets on different computers and platforms is custom dataset organization and file manipulation. Path structures are coded differently on windows and linux systems (/ vs \\) and datasets may be organized differently (e.g. keeping files together by session or by file type - audio together, video together, etc.).

Additionally, paths are different across hard drives and machines, which can lead to variable path switching in files that shouldn't contain path details. 

To improve working across platforms, this toolkit provides a function for generating a list of files from a configuration file, get_dirs_from_config. This function returns a list of direct file paths from the configuration, which can be used by the functions in the pipeline. A complete example of this is shown in step 1. Seperating Audio From Video. For now, take a look at the following example of how the support group dataset is configured on my personal computer:

```
dir_pattern:
  - /
  - media
  - chris
  - M2
  - 1-Raw_Data
  - Videos
  - dir1
  - raw_camera_output
  - dir2
```
substitutes:
```
dir1:
    - 1
    - 2
    - 3
```

```
dir2:
    - left.mp4
    - right.mp4
    - center.mp4
```

As can be seen in the next cell:


In [2]:
from pipeline.common.file_utils import get_dirs_from_config

# the provided sample config shows how the support group dataset is set up. It is useful as an example
# because it shows how the substitution works combinatorially 
get_dirs_from_config('../../pipeline/common/utils_config/dir_sample_config.yml')

['/media/chris/M2/1-Raw_Data/Videos/1/raw_camera_output/left.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/1/raw_camera_output/right.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/1/raw_camera_output/center.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/2/raw_camera_output/left.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/2/raw_camera_output/right.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/2/raw_camera_output/center.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/3/raw_camera_output/left.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/3/raw_camera_output/right.mp4',
 '/media/chris/M2/1-Raw_Data/Videos/3/raw_camera_output/center.mp4']

As you can see, this generates full paths for all the video files, which have been recorded on this dataset. We can generate a parallel output list of destination folders or files which we can then pass together to any of the appropriate functions in the pipeline (e.g. audio_from_video).

To modify this to work for a windows machine, one only has to change the base path to the appropriate directory (e.g. c:, path, to, dataset)

# 1. Seperating Audio From Video

Here I will show how to seperate the audio from video in our example. Since we only are using one video in this walkthrough, using the dataset file utilities may be overkill, but I will use them to show what it would be like for a larger dataset.

Many of the functions in the pipeline are simply wrappers on command line calls. These could also be accomplished with bash scripting, but because this is a python package we are including the wrappers as methods in the modules.

For example, audio_from_video() wraps the call ` $ffmpeg -i {src_video} -ac 1 {dst_audio}` to the command line

In [3]:
# Method using functions directly:
from pipeline.common.file_utils import audio_from_video

src_video = "../../data/video/1/short_example.mp4"
dst_audio = "../../data/audio/1/short_example.wav"

audio_from_video(src_video, dst_audio)

$ ffmpeg -i ../../data/video/1/short_example.mp4 -ac 1 ../../data/audio/1/short_example.wav


In [4]:
# Method using file utilities
from pipeline.common.file_utils import get_dirs_from_config
from pipeline.common.file_utils import audio_from_video

v_paths = get_dirs_from_config("./walkthrough_configs/video.yml")
a_paths = get_dirs_from_config("./walkthrough_configs/audio.yml")

for i in range(len(v_paths)):
    # NOTE FFMPEG sometimes fails to gel with os.popen calls in jupyter notebooks - just run this code in the terminal instead
    audio_from_video(v_paths[i], a_paths[i])


$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/1/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline/data/audio/1/short_example.wav
$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/2/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline/data/audio/2/short_example.wav
$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/3/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline/data/audio/3/short_example.wav
$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/4/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline/data/audio/4/short_example.wav
$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/5/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline/data/audio/5/short_example.wav
$ ffmpeg -i /home/interactionlab/chrisb/modeling-pipeline/data/video/6/short_example.mp4 -ac 1 /home/interactionlab/chrisb/modeling-pipeline

# 2. Feature Extraction

Here we will see how to extract audio and video features. Extracting audio features can be done with either Librosa or OpenSmile, which are available as pip packages. Extracting facial features can be done with OpenFace, which unfortunately requires a seperate installation. 

To be more efficient with file utility here, we will reuse the same feature pattern across the features we are looking for.

In [7]:
from pipeline.feature_extraction.librosa_features import get_librosa_features
from pipeline.feature_extraction.opensmile_features import get_opensmile_features
from pipeline.feature_extraction.facemesh_features import get_facemesh_features

# from pipeline.feature_extraction.openface_features import get_openface_features
# (remember to install openface first https://github.com/TadasBaltrusaitis/OpenFace/wiki/Unix-Installation)

openface_exe_path = "/home/chris/Programs/OpenFace/build/bin/FeatureExtraction"

v_paths = get_dirs_from_config("./walkthrough_configs/video.yml")
a_paths = get_dirs_from_config("./walkthrough_configs/audio.yml")
feat_paths = get_dirs_from_config("./walkthrough_configs/feats.yml")

for i in range(len(a_paths)):
    # It is easy to reuse the feature paths if they just point to a directory
    l = get_librosa_features(a_paths[i], feat_paths[i]+"/librosa_audio.csv")
    o = get_opensmile_features(a_paths[i], feat_paths[i]+"/opensmile_audio.csv")
    f = get_facemesh_features(v_paths[i], feat_paths[i]+"/mediapipe_facemesh.csv")
    
    # OpenFace must be installed prior to running. This demo is being created on
    # Ubuntu 20.04, which is not supported. So these features will not be used 
    # in this example.
    
    # On a system with openface installed, the following can be used:
    # get_openface_features(openface_exe_path, v_paths[i], feat_paths[i]+"/")


Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.


# 3. Annotation Generation

For this simple example I will not go through the available annotation options. Instead we will just use voice activity as our label. This could generally be considered a feature, but for this walkthrough we will use it as a label.

In addition, annotation expects the utterances json to have a label, so for this example will just edit the utterances.json `"speaker": ""` to be `"speaker":"speaking"`. This will make "speaking" our one hot label.

In [13]:
from pipeline.feature_extraction.voice_activity_features import get_voice_activity
from pipeline.annotation.annotations_to_csv import convert_annotations

a_paths = get_dirs_from_config("./walkthrough_configs/audio.yml")
va_paths = get_dirs_from_config("./walkthrough_configs/VA_annotations.yml")

for i in range(len(a_paths)):
    get_voice_activity(a_paths[i],0, va_paths[i])
    convert_annotations(va_paths[i]+"/utterances.json", va_paths[i]+"/speaking.csv", frame_rate=30, json_key="utterances",label_key="label")

/home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/audio/1/short_example.wav
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/00.wav 0.42000000000000015 2.0400000000000014
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/01.wav 2.100000000000001 3.2699999999999934
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/02.wav 3.659999999999991 6.900000000000013
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/03.wav 6.960000000000013 9.149999999999997
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/04.wav 20.340000000000014 27.720000000000294
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/1/chunks/05.wav 28.620000000000328 29.49000000000036

Labels include: 
 ['speaking']


Pandas Apply:   0%|          | 0/885 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/885 [00:00<?, ?it/s]

/home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/audio/2/short_example.wav
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/00.wav 7.320000000000016 8.010000000000021
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/01.wav 8.190000000000017 9.839999999999982
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/02.wav 9.92999999999998 10.769999999999962
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/03.wav 14.489999999999883 16.439999999999866
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/04.wav 17.63999999999991 18.329999999999938
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/05.wav 18.62999999999995 22.11000000000008
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/2/chunks/06.wav 22.830

Pandas Apply:   0%|          | 0/5931 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/7272 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7272 [00:00<?, ?it/s]

/home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/audio/13/short_example.wav
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/00.wav 1.260000000000001 3.149999999999994
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/01.wav 4.16999999999999 11.969999999999937
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/02.wav 12.239999999999931 13.049999999999914
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/03.wav 13.22999999999991 31.08000000000042
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/04.wav 33.2400000000005 37.20000000000065
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/05.wav 37.77000000000067 48.00000000000106
 Writing /home/interactionlab/chrisb/modeling-pipeline/data/walkthrough/va_output/13/chunks/06.wav 

Pandas Apply:   0%|          | 0/5670 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5670 [00:00<?, ?it/s]

# 4. Choosing Appropriate Features

Feature selection is often a manual process, in which the developer chooses features that they know either from experience or from the literature are most likely to be successful. There are more rigorous approaches however. One of these is to remove redundant features by looking at the level of feature correlation.

In this example, we use the function select_by_correlation() to get a list of the uncorrelated features from the csv of Librosa features we extracted earlier.

In [11]:
from pipeline.modeling.select_features import select_by_correlation

feat_paths = get_dirs_from_config("./walkthrough_configs/feats.yml")

for i in range(len(feat_paths)):
    l = select_by_correlation(feat_paths[i]+"/librosa_audio.csv")
    print("Librosa features to use:")
    for f in l:
        print(f)
    l = select_by_correlation(feat_paths[i]+"/mediapipe_facemesh.csv")
    print("facemesh features to use:")
    for f in l:
        print(f)
    l = select_by_correlation(feat_paths[i]+"/opensmile_audio.csv")
    print("opensmile features to use:")
    for f in l:
        print(f)
    break
    



Librosa features to use:
MFCC_0
MFCC_1
MFCC_2
MFCC_3
MFCC_4
MFCC_5
MFCC_6
MFCC_7
MFCC_8
MFCC_9
MFCC_10
MFCC_11
MFCC_12
MFCC_13
MFCC_14
MFCC_15
MFCC_16
MFCC_17
MFCC_18
MFCC_19
Chroma_0
Chroma_2
Chroma_3
Chroma_4
Chroma_5
Chroma_7
Chroma_9
Mel_Spect_0
Mel_Spect_1
Mel_Spect_2
Mel_Spect_4
Mel_Spect_21
Mel_Spect_30
Mel_Spect_32
Mel_Spect_34
Mel_Spect_36
Mel_Spect_38
Mel_Spect_40
Mel_Spect_41
Mel_Spect_42
Mel_Spect_44
Mel_Spect_46
Mel_Spect_50
Mel_Spect_52
Mel_Spect_54
Mel_Spect_56
Mel_Spect_58
Mel_Spect_60
Mel_Spect_65
Mel_Spect_73
Mel_Spect_91
Spect_Contrast_0
Spect_Contrast_1
Spect_Contrast_2
Spect_Contrast_3
Spect_Contrast_4
Spect_Contrast_5
Spect_Contrast_6
Rolloff_0
facemesh features to use:
x_0
y_0
y_4
y_6
y_66
opensmile features to use:
Loudness_sma3
alphaRatio_sma3
hammarbergIndex_sma3
slope0-500_sma3
slope500-1500_sma3
mfcc1_sma3
mfcc2_sma3
mfcc3_sma3
F0semitoneFrom27.5Hz_sma3nz
jitterLocal_sma3nz
shimmerLocaldB_sma3nz
HNRdBACF_sma3nz
logRelF0-H1-H2_sma3nz
logRelF0-H1-A3_sma3nz
F1f

This feature list is a subset of the total features we extracted.

**Before we continue onto the next step**, we should insert this list of features into the data loader configuration file as is shown in "./walkthrough_configs/data_loader_pearson_config.yml"

The data loader uses a configuration similar to the one we have seen so far, but with some extensions. We use the same format for specifying files, with a directory list and substitutions, however these are nested within feature set labels. Matching these labels we have a list of the features we wish to use, which are available as columns in each of the feature files. We will be using the features we found were uncorrelated in step 4, as well as the annotations. The annotation labels are not treated specially here, and are specified just like the features.

# Next Steps

Head over to the walkthrough_part_2.ipynb to see how to train a model!